In [1]:
# import packages
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import anndata
#from gprofiler import gprofiler
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri
from scipy.spatial import distance
import logging
import warnings
warnings.filterwarnings("ignore")

# For >= python 3.9, you need to run "mp.set_start_method('spawn')"
# or multiprocessing will not function properly with ipython
# Windows users may need to change 'spawn' to 'sys.platform'
# Running this line in =< python 3.8 is fine but not necessary
# Here I'm using python 3.8 but still running this line for completeness
import multiprocessing as mp
mp.set_start_method('spawn')

# declare settings
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

# declare scanpy settings
plt.rcParams['figure.figsize']=(7,7)
sc.set_figure_params(dpi=150, dpi_save=300)
sc.settings.verbosity = 0
sc.logging.print_versions()

-----
anndata     0.9.1
scanpy      1.9.3
-----
PIL                 9.5.0
anndata2ri          1.1
anyio               NA
argcomplete         NA
arrow               1.2.3
asttokens           NA
attr                23.1.0
babel               2.12.1
backcall            0.2.0
brotli              NA
certifi             2023.05.07
cffi                1.15.1
charset_normalizer  3.1.0
colorama            0.4.6
comm                0.1.3
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.7
decorator           5.1.1
defusedxml          0.7.1
executing           1.2.0
fastjsonschema      NA
fqdn                NA
gmpy2               2.1.2
google              NA
h5py                3.9.0
idna                3.4
igraph              0.10.4
ipykernel           6.23.2
ipython_genutils    0.2.0
isoduration         NA
jedi                0.18.2
jinja2              3.1.2
joblib              1.2.0
json5               NA
jsonpointer         2.0
jsonschema    

In [2]:
adata = sc.datasets.pbmc3k()

In [3]:
adata

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [4]:
# Adata object contains raw counts
np.unique(adata.X.A.flatten())

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [5]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
print('Normalizing...')
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e5)
sc.pp.log1p(adata_pp)
print('PCA...')
sc.pp.pca(adata_pp, n_comps=15)
print('Neighbors...')
sc.pp.neighbors(adata_pp)
print('Clustering...')
sc.tl.leiden(adata_pp, key_added='groups', resolution=0.8)

Normalizing...
PCA...
Neighbors...
Clustering...


In [6]:
def merge_small_clusters(adata_pp, min_cluster_size):
    
    while not (all(clust_size >= min_cluster_size for clust_size in adata_pp.obs.groups.value_counts() if clust_size != 0)):
    
        c_counts = adata_pp.obs.groups.value_counts()
        cluster_list = list(c_counts.keys())

        merge_dict = {}

        for c in cluster_list:

            if c_counts[c] < min_cluster_size:

                #find closets cluster
                avg_distances = {}
                k_clusts = [l for l in cluster_list if l!=c]
                for k in k_clusts:
                    c_cells = adata_pp[adata_pp.obs['groups']==c,:].obsm['X_pca']
                    k_cells = adata_pp[adata_pp.obs['groups']==k,:].obsm['X_pca']
                    dist_mat = distance.cdist(c_cells, k_cells, metric='cosine')
                    mean_val = np.mean(dist_mat[np.triu_indices(n=len(c_cells), m=len(k_cells) , k = 1)])
                    avg_distances[k] = mean_val

                min_k = min(avg_distances, key=avg_distances.get)
                merge_dict[c]=min_k

        for c in list(merge_dict.keys()):
            adata_pp.obs.loc[adata_pp.obs['groups']==c,'groups'] = merge_dict[c]
        
    # reset categories
    adata_pp.obs['groups'] = adata_pp.obs['groups'].astype('str').astype('category')
    
    return adata_pp

In [7]:
merge_small_clusters(adata_pp, 100)

AnnData object with n_obs × n_vars = 2700 × 32738
    obs: 'n_counts', 'groups'
    var: 'gene_ids'
    uns: 'log1p', 'pca', 'neighbors', 'leiden'
    obsm: 'X_pca'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [8]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
adata.obs['norm_groups'] = input_groups
adata.obs['norm_groups'].value_counts()

norm_groups
0    501
1    484
2    372
3    334
4    301
6    236
5    196
7    154
8    122
Name: count, dtype: int64

In [9]:
adata

AnnData object with n_obs × n_vars = 2700 × 32738
    obs: 'norm_groups'
    var: 'gene_ids'

In [10]:
import scranPY
scranPY.compute_sum_factors(adata, clusters='norm_groups', parallelize=True, algorithm='CVXPY', max_size=3000, plotting=True,
    lower_bound=0.4, normalize_counts=False)


Current smallest cluster =  122  cells.
Using max_size =  3000 , clusters have been split into  9  clusters.


ValueError: operands could not be broadcast together with shapes (32738,2700) (2700,1) 